<a href="https://colab.research.google.com/github/mobilemie/ConvertExceltoPdf/blob/main/ML_for_lab_matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxlsb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [112]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [114]:
pd.set_option('display.max_columns', 700)

In [4]:
df1 = pd.read_excel('/content/drive/My Drive/final_lab_mockup.xlsb' ,sheet_name= 'TMLT')
df2 = pd.read_excel('/content/drive/My Drive/final_lab_mockup.xlsb' ,sheet_name= 'THCC')
df3 = pd.read_excel('/content/drive/My Drive/final_lab_mockup.xlsb' ,sheet_name= 'CDG')
df4 = pd.read_excel('/content/drive/My Drive/final_lab_mockup.xlsb' ,sheet_name= 'LOINC')

In [5]:
df1 = df1.rename(columns= {'TMLT_Code':'code', 'TMLT_Name':'input name'})
df1.head()

,full name,panel name,code,input name
0,NaN,Complete Blood Count,300082,Complete blood count panel in Blood by Automat...
1,NaN,Serum Electrolytes,320258,"Electrolyte panel (Na, K, Cl, CO2) in Serum or..."
2,NaN,Lipid Profile,320259,"Lipid profile (Cholesterol, HDL-chol, LDL-cho..."
3,NaN,Liver Function Test,320261,Liver function test in Serum or Plasma
4,Hemoglobin,Complete Blood Count,300089,Hemoglobin [g/dL] in Blood


In [6]:
df2 = df2.rename(columns= {'THCC_Code':'code', 'THCC_Name_EN':'input name'})
df2 = df2.drop(columns=['THCC_Name_TH'])
df2.head()

,full name,panel name,code,input name
0,Hemoglobin,Complete Blood Count,621401.0,"Hemoglobin, whole blood"
1,Hemoglobin,Complete Blood Count,621402.0,"Hemoglobin, plasma"
2,Hematocrit,Complete Blood Count,621201.0,"Hematocrit, whole blood"
3,Neutrophil,Complete Blood Count,622201.0,"White blood cell count, whole blood"
4,Absolute Neutrophil Count,Complete Blood Count,622201.0,"White blood cell count, whole blood"


In [7]:
df3 = df3.rename(columns= {'CGD_CODE':'code', 'CGD_Name':'input name'})
df3.head()

,full name,panel name,code,input name
0,NaN,Complete Blood Count,30102.0,Complete blood count without smear
1,NaN,Serum Electrolytes,32001.0,"Electrolyte (Na, K, Cl, CO2)"
2,NaN,Lipid Profile,32004.0,"Lipid profile (Cholesterol, HDL-chol, LDL-ch..."
3,NaN,Liver Function Test,32003.0,Liver function test
4,Hemoglobin,Complete Blood Count,NaN,NaN


In [8]:
df4 = df4.rename(columns= {'LOINC_NUM':'code', 'LOINC_LONG_COMMON_NAME':'input name'})
df4.head()

,full name,panel name,code,input name
0,Hemoglobin,Complete Blood Count,718-7,Hemoglobin [Mass/volume] in Blood
1,Hematocrit,Complete Blood Count,4544-3,Hematocrit [Volume Fraction] of Blood by Autom...
2,Hematocrit,Complete Blood Count,4545-0,Hematocrit [Volume Fraction] of Blood by Centr...
3,Hematocrit,Complete Blood Count,31100-1,Hematocrit [Volume Fraction] of Blood by Imped...
4,Neutrophil,Complete Blood Count,770-8,Neutrophils/100 leukocytes in Blood by Automat...


In [9]:
res = pd.concat([df1, df2, df3, df4], keys=["tmlt", "thcc", "cdg", "loinc"])

In [10]:
res = res.fillna(value= "Null")

In [11]:
res.head()

full name            panel name    code  \
tmlt 0        Null  Complete Blood Count  300082   
     1        Null    Serum Electrolytes  320258   
     2        Null         Lipid Profile  320259   
     3        Null   Liver Function Test  320261   
     4  Hemoglobin  Complete Blood Count  300089   

                                               input name  
tmlt 0  Complete blood count panel in Blood by Automat...  
     1  Electrolyte panel (Na, K, Cl, CO2) in Serum or...  
     2  Lipid profile (Cholesterol, HDL-chol,  LDL-cho...  
     3             Liver function test in Serum or Plasma  
     4                         Hemoglobin [g/dL] in Blood

In [12]:
splitInputName = res.loc[:, ('input name')].str.lower().str.split(r"[\([{ });,\]]+")
res['split input name'] = splitInputName
res.head()

full name            panel name    code  \
tmlt 0        Null  Complete Blood Count  300082   
     1        Null    Serum Electrolytes  320258   
     2        Null         Lipid Profile  320259   
     3        Null   Liver Function Test  320261   
     4  Hemoglobin  Complete Blood Count  300089   

                                               input name  \
tmlt 0  Complete blood count panel in Blood by Automat...   
     1  Electrolyte panel (Na, K, Cl, CO2) in Serum or...   
     2  Lipid profile (Cholesterol, HDL-chol,  LDL-cho...   
     3             Liver function test in Serum or Plasma   
     4                         Hemoglobin [g/dL] in Blood   

                                         split input name  
tmlt 0  [complete, blood, count, panel, in, blood, by,...  
     1  [electrolyte, panel, na, k, cl, co2, in, serum...  
     2  [lipid, profile, cholesterol, hdl-chol, ldl-ch...  
     3     [liver, function, test, in, serum, or, plasma]  
     4                      [hemoglobin, g/dl, in, blood]

In [115]:
res['panel and full name'] = res['panel name'] + ';' +  res['full name']
res.head()

full name            panel name    code  \
tmlt 0        Null  Complete Blood Count  300082   
     1        Null    Serum Electrolytes  320258   
     2        Null         Lipid Profile  320259   
     3        Null   Liver Function Test  320261   
     4  Hemoglobin  Complete Blood Count  300089   

                                               input name  \
tmlt 0  Complete blood count panel in Blood by Automat...   
     1  Electrolyte panel (Na, K, Cl, CO2) in Serum or...   
     2  Lipid profile (Cholesterol, HDL-chol,  LDL-cho...   
     3             Liver function test in Serum or Plasma   
     4                         Hemoglobin [g/dL] in Blood   

                                         split input name  \
tmlt 0  [complete, blood, count, panel, in, blood, by,...   
     1  [electrolyte, panel, na, k, cl, co2, in, serum...   
     2  [lipid, profile, cholesterol, hdl-chol, ldl-ch...   
     3     [liver, function, test, in, serum, or, plasma]   
     4                      [hemoglobin, g/dl, in, blood]   

                    full and panel name              panel and full name  
tmlt 0        Complete Blood Count;Null        Complete Blood Count;Null  
     1          Serum Electrolytes;Null          Serum Electrolytes;Null  
     2               Lipid Profile;Null               Lipid Profile;Null  
     3         Liver Function Test;Null         Liver Function Test;Null  
     4  Complete Blood Count;Hemoglobin  Complete Blood Count;Hemoglobin

In [ ]:
#encode label
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
encoded_y = label_encoder.fit_transform(res['full and panel name'])
label_encoder_name_mapping = dict(zip(label_encoder.classes_,
                                         label_encoder.transform(label_encoder.classes_)))
# label_encoder_name_mapping
pd.DataFrame(encoded_y)

In [14]:
text = res['split input name']
count_vectorizer = CountVectorizer(tokenizer= lambda x:x, lowercase = False)
bag_of_words = count_vectorizer.fit_transform(text)
bag_of_words = pd.DataFrame(bag_of_words.toarray(),
				columns = count_vectorizer.get_feature_names(), index=res.index)
bag_of_words.head()

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


#/area  #/volume  %  +  +/-  -  --1  --2  /ml  /mm3  0.5  1  1+2  \
tmlt 0  0       0         0  0  0    0  0    0    0    0     0    0  0    0   
     1  0       0         0  0  0    0  0    0    0    0     0    0  0    0   
     2  1       0         0  0  0    0  0    0    0    0     0    0  0    0   
     3  0       0         0  0  0    0  0    0    0    0     0    0  0    0   
     4  0       0         0  0  0    0  0    0    0    0     0    0  0    0   

        1.7  1.9  10*3/ul  10*6/ul  10*9/l  10^12/l  10^3/ul  2  2.6  20  \
tmlt 0    0    0        0        0       0        0        0  0    0   0   
     1    0    0        0        0       0        0        0  0    0   0   
     2    0    0        0        0       0        0        0  0    0   0   
     3    0    0        0        0       0        0        0  0    0   0   
     4    0    0        0        0       0        0        0  0    0   0   

        2019  24  25  400  50  75  =  ?g/dl  a1c  a1c/hemoglobin.total  ab  \
tmlt 0     0   0   0    0   0   0  0      0    0                     0   0   
     1     0   0   0    0   0   0  0      0    0                     0   0   
     2     0   0   0    0   0   0  0      0    0                     0   0   
     3     0   0   0    0   0   0  0      0    0                     0   0   
     4     0   0   0    0   0   0  0      0    0                     0   0   

        ab+hiv1  acetylcholine  acid  activated  active  activity  \
tmlt 0        0              0     0          0       0         0   
     1        0              0     0          0       0         0   
     2        0              0     0          0       0         0   
     3        0              0     0          0       0         0   
     4        0              0     0          0       0         0   

        activity/volume  actual/normal  aerobe  aerobic  ag  alanine  albumin  \
tmlt 0                0              0       0        0   0        0        0   
     1                0              0       0        0   0        0        0   
     2                0              0       0        0   0        0        0   
     3                0              0       0        0   0        0        0   
     4                0              0       0        0   0        0        0   

        alkaline  alt  aminotransferase  amplification  ana  anaerobe  \
tmlt 0         0    0                 0              0    0         0   
     1         0    0                 0              0    0         0   
     2         0    0                 0              0    0         0   
     3         0    0                 0              0    0         0   
     4         0    0                 0              0    0         0   

        anaerobic  and  anti  anti-cardiolipin  anti-dsdna  anti-sm  antibody  \
tmlt 0          0    0     0                 0           0        0         0   
     1          0    0     0                 0           0        0         0   
     2          0    0     0                 0           0        0         0   
     3          0    0     0                 0           0        0         0   
     4          0    0     0                 0           0        0         0   

        anticardiolipin  antigen  antiglobulin  antinuclear  any  appearance  \
tmlt 0                0        0             0            0    0           0   
     1                0        0             0            0    0           0   
     2                0        0             0            0    0           0   
     3                0        0             0            0    0           0   
     4                0        0             0            0    0           0   

        aptt  arterial  aspartate  assay  ast  automate  automated  b  \
tmlt 0     0         0          0      0    0         0          1  0   
     1     0         0          0      0    0         0          0  0   
     2     0         0          0      0    0         0          0  0   
     3   

In [16]:
#tfidf
tfidf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf = tfidf_transformer.fit_transform(bag_of_words)
result = pd.DataFrame(tfidf.toarray(), columns = count_vectorizer.get_feature_names())
print(result)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


                #/area  #/volume    %    +  +/-    -  --1  --2  /ml  /mm3  \
0     0.000000     0.0       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   
1     0.000000     0.0       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   
2     0.193901     0.0       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   
3     0.000000     0.0       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   
4     0.000000     0.0       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   
...        ...     ...       ...  ...  ...  ...  ...  ...  ...  ...   ...   
1005  0.000000     0.0       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   
1006  0.000000     0.0       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   
1007  0.000000     0.0       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   
1008  0.000000     0.0       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   
1009  0.000000     0.0       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0   

      0.5    1  1+2  1.7  1.9  10*3/ul  10*6/ul  10*9/l  10^12/l  10^3/ul  

In [ ]:
# res.groupby(['panel name'], group_keys=True).mean()
res.groupby(['panel name'], group_keys=True).count()

In [ ]:
res.groupby(['panel name', 'full name'], group_keys=True).count()

In [19]:
data1 = pd.DataFrame(res['full name'])
data2 = pd.DataFrame(res['panel name'])

In [ ]:
newBag = pd.concat([data1, data2, bag_of_words])
newBag.sample(10)

In [21]:
#แบ่ง panel name เป็น train test
# x = res['split input name']
# y = res['panel name']
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [127]:
x = res['split input name']
y = res['panel and full name']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [ ]:
#bag of word ของ full and panel name
cv = CountVectorizer(tokenizer= lambda x:x, lowercase = False) 
word_count_vector = cv.fit_transform(x_train)
word_count_vector = pd.DataFrame(word_count_vector.toarray(),
				columns = cv.get_feature_names(), index=x_train.index)
word_count_vector

In [123]:
#x_train count
count_vec = CountVectorizer(tokenizer= lambda x:x, lowercase = False)
bowX = count_vec.fit_transform(x_train)
bowX = pd.DataFrame(bowX.toarray(),
				columns = count_vec.get_feature_names(), index=x_train.index)
bowX

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


#/area  #/volume  %  +  +/-  -  --1  --2  /mm3  0.5  1  1+2  \
loinc 62   0       0         0  0  0    0  0    0    0     0    0  0    0   
      25   0       0         0  0  0    0  0    0    0     0    0  0    0   
cdg   171  0       0         0  0  0    0  0    0    0     0    0  0    0   
tmlt  150  0       0         0  0  0    0  0    0    0     0    0  1    0   
cdg   43   0       0         0  0  0    0  0    0    0     0    0  0    0   
...       ..     ...       ... .. ..  ... ..  ...  ...   ...  ... ..  ...   
loinc 132  0       0         0  0  0    0  0    0    0     0    0  0    0   
tmlt  192  0       0         0  0  0    0  0    0    0     0    0  0    0   
cdg   142  0       0         0  0  0    0  0    0    0     0    0  0    0   
      72   1       0         0  0  0    0  1    0    0     0    0  0    0   
      197  1       0         0  0  0    0  0    0    0     0    0  0    0   

           1.7  1.9  10*3/ul  10*6/ul  10*9/l  10^12/l  10^3/ul  2  2.6  20  \
loinc 62     0    0        0        0       0        0        0  0    0   0   
      25     0    0        0        0       0        0        0  0    0   0   
cdg   171    0    0        0        0       0        0        0  0    0   0   
tmlt  150    0    0        0        0       0        0        0  0    0   0   
cdg   43     0    0        0        0       0        0        0  0    0   0   
...        ...  ...      ...      ...     ...      ...      ... ..  ...  ..   
loinc 132    0    0        0        0       0        0        0  0    0   0   
tmlt  192    0    0        0        0       0        0        0  0    0   0   
cdg   142    0    0        0        0       0        0        0  0    0   0   
      72     0    0        0        0       0        0        0  0    0   0   
      197    0    0        0        0       0        0        0  0    0   0   

           2019  24  25  400  50  75  =  ?g/dl  a1c  a1c/hemoglobin.total  ab  \
loinc 62      0   0   0    0   0   0  0      0    0                     0   0   
      25      0   0   0    0   0   0  0      0    0                     0   0   
cdg   171     0   0   0    0   0   0  0      0    0                     0   0   
tmlt  150     0   0   0    0   0   0  0      0    0                     0   0   
cdg   43      0   0   0    0   0   0  0      0    0                     0   0   
...         ...  ..  ..  ...  ..  .. ..    ...  ...                   ...  ..   
loinc 132     0   0   0    0   0   0  0      0    0                     0   0   
tmlt  192     0   0   0    0   0   0  0      0    0                     0   1   
cdg   142     0   0   0    0   0   0  0      0    0                     0   0   
      72      0   0   0    0   0   0  0      0    0                     0   0   
      197     0   0   0    0   0   0  0      0    0                     0   0   

           ab+hiv1  acetylcholine  acid  activated  active  activity  \
loinc 62         0              0     0          0       0         0   
      25         0              0     0          0       0         0   
cdg   171        0              0     0          0       0         0   
tmlt  150        0              0     0          0       0         0   
cdg   43         0              0     0          0       0         0   
...            ...            ...   ...        ...     ...       ...   
loinc 132        0              0     0          0       0         0   
tmlt  192        0              0     0          0       0         0   
cdg   142        0              0     0          0       0         0   
      72         0              0     0          0       0         0   
      197        0              0     0          0       0         0   

           activity/volume  actual/normal  aerobe  aerobic  ag  alanine  \
loinc 62                 0              0       0        0   0        0   
      25                 0              0       0        0   0        0   
cdg   171                0              0       0        0   0        0   
tm

In [23]:
#x_test count
bowXt = count_vec.fit_transform(x_test)
bowXt = pd.DataFrame(bowXt.toarray(),
				columns = count_vec.get_feature_names(), index=x_test.index)

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier 
from sklearn import metrics

In [25]:
#knn bag of word
knn = KNeighborsClassifier(n_neighbors = 5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=-1)
knn.fit(bowX, y_train)
knn.score(bowX,y_train)

0.7289603960396039

In [26]:
df_newdata = bowXt.reindex(labels=bowX.columns,axis=1)
df_newdata = df_newdata.fillna(value= 0)
df_newdata

#/area  #/volume  %    +  +/-  -  --1  --2  /mm3  0.5  1  1+2  \
loinc 98   0       0         0  0  0.0    0  0  0.0  0.0     0  0.0  0    0   
thcc  0    0       0         0  0  0.0    0  0  0.0  0.0     0  0.0  0    0   
tmlt  85   0       0         0  0  0.0    0  0  0.0  0.0     0  0.0  0    0   
thcc  124  0       0         0  0  0.0    0  0  0.0  0.0     0  0.0  0    0   
tmlt  204  0       0         0  0  0.0    1  0  0.0  0.0     0  0.0  0    0   
...       ..     ...       ... ..  ...  ... ..  ...  ...   ...  ... ..  ...   
loinc 81   0       0         1  0  0.0    0  0  0.0  0.0     0  0.0  0    0   
tmlt  231  0       0         0  0  0.0    0  0  0.0  0.0     0  0.0  0    0   
      5    0       0         0  1  0.0    0  0  0.0  0.0     0  0.0  0    0   
thcc  141  0       0         0  0  0.0    0  0  0.0  0.0     0  0.0  0    0   
loinc 306  0       0         0  0  0.0    0  0  0.0  0.0     0  0.0  0    0   

           1.7  1.9  10*3/ul  10*6/ul  10*9/l  10^12/l  10^3/ul  2  2.6  20  \
loinc 98     0  0.0        0      0.0     0.0      0.0      0.0  0  0.0   0   
thcc  0      0  0.0        0      0.0     0.0      0.0      0.0  0  0.0   0   
tmlt  85     0  0.0        0      0.0     0.0      0.0      0.0  0  0.0   0   
thcc  124    0  0.0        0      0.0     0.0      0.0      0.0  0  0.0   0   
tmlt  204    0  0.0        0      0.0     0.0      0.0      0.0  0  0.0   0   
...        ...  ...      ...      ...     ...      ...      ... ..  ...  ..   
loinc 81     0  0.0        0      0.0     0.0      0.0      0.0  0  0.0   0   
tmlt  231    0  0.0        0      0.0     0.0      0.0      0.0  0  0.0   0   
      5      0  0.0        0      0.0     0.0      0.0      0.0  0  0.0   0   
thcc  141    0  0.0        0      0.0     0.0      0.0      0.0  0  0.0   0   
loinc 306    0  0.0        0      0.0     0.0      0.0      0.0  0  0.0   0   

           2019  24   25  400   50   75  =  ?g/dl  a1c  a1c/hemoglobin.total  \
loinc 98    0.0   0  0.0    0  0.0  0.0  0    0.0  0.0                   0.0   
thcc  0     0.0   0  0.0    0  0.0  0.0  0    0.0  0.0                   0.0   
tmlt  85    0.0   0  0.0    0  0.0  0.0  0    0.0  0.0                   0.0   
thcc  124   0.0   0  0.0    0  0.0  0.0  0    0.0  0.0                   0.0   
tmlt  204   0.0   0  0.0    0  0.0  0.0  0    0.0  0.0                   0.0   
...         ...  ..  ...  ...  ...  ... ..    ...  ...                   ...   
loinc 81    0.0   0  0.0    0  0.0  0.0  0    0.0  0.0                   0.0   
tmlt  231   0.0   0  0.0    0  0.0  0.0  0    0.0  0.0                   0.0   
      5     0.0   0  0.0    0  0.0  0.0  0    0.0  0.0                   0.0   
thcc  141   0.0   0  0.0    0  0.0  0.0  0    0.0  0.0                   0.0   
loinc 306   0.0   0  0.0    0  0.0  0.0  0    0.0  0.0                   0.0   

           ab  ab+hiv1  acetylcholine  acid  activated  active  activity  \
loinc 98    0      0.0              0   0.0        0.0     0.0       0.0   
thcc  0     0      0.0              0   0.0        0.0     0.0       0.0   
tmlt  85    0      0.0              0   0.0        0.0     0.0       0.0   
thcc  124   0      0.0              0   0.0        0.0     0.0       0.0   
tmlt  204   1      0.0              0   0.0        0.0     0.0       0.0   
...        ..      ...            ...   ...        ...     ...       ...   
loinc 81    0      0.0              0   0.0        0.0     0.0       0.0   
tmlt  231   0      0.0              0   0.0        0.0     0.0       0.0   
      5     0      0.0              0   0.0        0.0     0.0       0.0   
thcc  141   0      0.0              0   0.0        0.0     0.0       0.0   
loinc 306   0      0.0              0   0.0        0.0     0.0       0.0   

           activity/volume  actual/normal  aerobe  aerobic  ag  alanine  \
loinc 98                 0            0.0       0      0.0   1        0   
thcc  0                  0            0.0       0      0.0   0        0   
tmlt  85         

In [27]:
y_pred = knn.predict(df_newdata)
print("Accuracy KNN CountVect:",metrics.accuracy_score(y_test, y_pred))

Accuracy KNN CountVect: 0.5594059405940595


In [29]:
from sklearn.model_selection import cross_validate
from sklearn import model_selection
from sklearn.model_selection import KFold, cross_val_score
k_folds = KFold(n_splits = 5)
scores = cross_val_score(knn, bowX, y_train , cv = k_folds)
# scores = cross_validate(knn, bowX, y_train , scoring='accuracy', cv = k_folds, return_estimator=True)
# print(scores['fit_time'].mean())
print(scores.mean())

0.5780768345985737


In [30]:
#x_train 
countVec = CountVectorizer(tokenizer= lambda x:x, lowercase = False)
x_train_tfidf = tfidf_transformer.fit_transform(bowX)
# x_train_result = pd.DataFrame(x_train_tfidf.toarray(), columns = countVec.get_feature_names())
# x_train_result
print(x_train_tfidf.toarray())

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.22955729 0.         0.         ... 0.         0.         0.        ]
 [0.33929764 0.         0.         ... 0.         0.         0.        ]]


In [31]:
#idf
tsidf = countVec.fit_transform(x_train)
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(tsidf)
df_idf = pd.DataFrame(tfidf_transformer.idf_ , index=countVec.get_feature_names(),columns=["idf_weights"]) 
# df_idf.sort_values(by=['idf_weights'])
df_idf

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,idf_weights
,3.110831
#/area,7.002652
#/volume,4.199291
%,4.805427
+,7.002652
...,...
whole,4.700067
width,6.086361
with,4.560305
xxx,6.597187


In [84]:
#knn
clf = knn.fit(x_train_tfidf, y_train)
knn.score(x_train_tfidf,y_train)

0.7834158415841584

In [85]:
y_pred_tfidf = clf.predict(df_newdata)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but KNeighborsClassifier was fitted without feature names
  warnings.warn(


In [34]:
print("Accuracy KNN tf-idf:",metrics.accuracy_score(y_test, y_pred_tfidf))
	
print('Recall:' ,metrics.recall_score(y_test, y_pred_tfidf, average='macro'))

print("F1 Score:" , metrics.f1_score(y_test, y_pred_tfidf, average='macro'))

print('Precision:' ,metrics.precision_score(y_test, y_pred_tfidf, average='macro'))

Accuracy KNN tf-idf: 0.6336633663366337
Recall: 0.6135090491945331
F1 Score: 0.6119053780932663
Precision: 0.6400155958825262


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
df_x_test = pd.DataFrame(x_test)
df_x_train = pd.DataFrame(x_train)

In [36]:
df_x_test['split input name'].apply(len) 

loinc  98     12
thcc   0       3
tmlt   85     10
thcc   124     3
tmlt   204    11
              ..
loinc  81     10
tmlt   231     8
       5       7
thcc   141     4
loinc  306     8
Name: split input name, Length: 202, dtype: int64

In [37]:
all_bowXt = bowX.sum(axis =1)
all_bowXt

loinc  62      3
       25      8
cdg    171     1
tmlt   150    16
cdg    43      1
              ..
loinc  132     8
tmlt   192     7
cdg    142     1
       72      9
       197     6
Length: 808, dtype: int64

In [38]:
#tf
tf = bowX.T / all_bowXt.T
tf_ans = tf.T
tf_ans

#/area  #/volume    %    +  +/-         -  --1  --2  \
loinc 62   0.000000     0.0       0.0  0.0  0.0  0.0  0.000000  0.0  0.0   
      25   0.000000     0.0       0.0  0.0  0.0  0.0  0.000000  0.0  0.0   
cdg   171  0.000000     0.0       0.0  0.0  0.0  0.0  0.000000  0.0  0.0   
tmlt  150  0.000000     0.0       0.0  0.0  0.0  0.0  0.000000  0.0  0.0   
cdg   43   0.000000     0.0       0.0  0.0  0.0  0.0  0.000000  0.0  0.0   
...             ...     ...       ...  ...  ...  ...       ...  ...  ...   
loinc 132  0.000000     0.0       0.0  0.0  0.0  0.0  0.000000  0.0  0.0   
tmlt  192  0.000000     0.0       0.0  0.0  0.0  0.0  0.000000  0.0  0.0   
cdg   142  0.000000     0.0       0.0  0.0  0.0  0.0  0.000000  0.0  0.0   
      72   0.111111     0.0       0.0  0.0  0.0  0.0  0.111111  0.0  0.0   
      197  0.166667     0.0       0.0  0.0  0.0  0.0  0.000000  0.0  0.0   

           /mm3  0.5       1  1+2  1.7  1.9  10*3/ul  10*6/ul  10*9/l  \
loinc 62    0.0  0.0  0.0000  0.0  0.0  0.0      0.0      0.0     0.0   
      25    0.0  0.0  0.0000  0.0  0.0  0.0      0.0      0.0     0.0   
cdg   171   0.0  0.0  0.0000  0.0  0.0  0.0      0.0      0.0     0.0   
tmlt  150   0.0  0.0  0.0625  0.0  0.0  0.0      0.0      0.0     0.0   
cdg   43    0.0  0.0  0.0000  0.0  0.0  0.0      0.0      0.0     0.0   
...         ...  ...     ...  ...  ...  ...      ...      ...     ...   
loinc 132   0.0  0.0  0.0000  0.0  0.0  0.0      0.0      0.0     0.0   
tmlt  192   0.0  0.0  0.0000  0.0  0.0  0.0      0.0      0.0     0.0   
cdg   142   0.0  0.0  0.0000  0.0  0.0  0.0      0.0      0.0     0.0   
      72    0.0  0.0  0.0000  0.0  0.0  0.0      0.0      0.0     0.0   
      197   0.0  0.0  0.0000  0.0  0.0  0.0      0.0      0.0     0.0   

           10^12/l  10^3/ul    2  2.6   20  2019   24   25  400   50   75  \
loinc 62       0.0      0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0   
      25       0.0      0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0   
cdg   171      0.0      0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0   
tmlt  150      0.0      0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0   
cdg   43       0.0      0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0   
...            ...      ...  ...  ...  ...   ...  ...  ...  ...  ...  ...   
loinc 132      0.0      0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0   
tmlt  192      0.0      0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0   
cdg   142      0.0      0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0   
      72       0.0      0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0   
      197      0.0      0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0   

             =  ?g/dl  a1c  a1c/hemoglobin.total        ab  ab+hiv1  \
loinc 62   0.0    0.0  0.0                   0.0  0.000000      0.0   
      25   0.0    0.0  0.0                   0.0  0.000000      0.0   
cdg   171  0.0    0.0  0.0                   0.0  0.000000      0.0   
tmlt  150  0.0    0.0  0.0                   0.0  0.000000      0.0   
cdg   43   0.0    0.0  0.0                   0.0  0.000000      0.0   
...        ...    ...  ...                   ...       ...      ...   
loinc 132  0.0    0.0  0.0                   0.0  0.000000      0.0   
tmlt  192  0.0    0.0  0.0                   0.0  0.142857      0.0   
cdg   142  0.0    0.0  0.0                   0.0  0.000000      0.0   
      72   0.0    0.0  0.0                   0.0  0.000000      0.0   
      197  0.0    0.0  0.0                   0.0  0.000000      0.0   

           acetylcholine  acid  activated  active  activity  activity/volume  \
loinc 62             0.0   0.0        0.0     0.0       0.0              0.0   
      25             0.0   0.0        0.0     0.0       0.0              0.0   
cdg   171            0.0   0.0        0.0     0.0       0.0              0.0   
tmlt  150            0.0   0.0        0.0     0.0       0.0              0.0   
cdg   43             0.0   0.0        0.0     0.0    

In [39]:
new_idf = df_idf.T
new_idf

,,#/area,#/volume,%,+,+/-,-,--1,--2,/mm3,0.5,1,1+2,1.7,1.9,10*3/ul,10*6/ul,10*9/l,10^12/l,10^3/ul,2,2.6,20,2019,24,25,400,50,75,=,?g/dl,a1c,a1c/hemoglobin.total,ab,ab+hiv1,acetylcholine,acid,activated,active,activity,activity/volume,actual/normal,aerobe,aerobic,ag,alanine,albumin,alkaline,alt,aminotransferase,amplification,anaerobe,and,anti,anti-dsdna,anti-sm,antibody,anticardiolipin,antigen,antiglobulin,antinuclear,any,appearance,aptt,arterial,aspartate,assay,ast,automate,automated,b,bacteria,bacterial,basophils,bcp,bicarbonate,bilirubin,bilirubin.albumin,bilirubin.direct,bilirubin.glucuronidated,bilirubin.total,binding,blood,bnp,body,bound,bromocresol,bun,by,c,c-reactive,c3,c4,calciferol,calcitonin,calcitriol,calcium,calculation,capacity,capillaries,capillary,cardiolipin,cd3+cd4+,cd4+cd8+,cell,cell/hpf,cells,cells/100,cells/l,cells/ml,cells/mm3,cells/ul,centrifugation,centrifuged,cerebral,chloride,chol,cholesterol,chylomicrons,ck-mb,ckd-epi,cl,clearance,clearance/1.73,co2,coagulation,collected,color,complement,complete,concentration,conjugated,control,copies/ml,cord,core,corona,coronavirus,corpuscular,cortisol,count,covid-19,cpk,creatine,creatinine,creatinine-based,crp,csf,culture,d,d+metabolites,d-dimer,dehydrogenase,dengue,deprecated,detection,differential,direct,disease,distribution,dna,double,dried,duration,dye,e,eclia,egfr,electrolyte,elisa,entitic,enzymatic,eosinophils,eosinophils/100,epithelial,erythrocyte,erythrocytes,erythrocytes/100,esr,esterase,estimates,examination,extractable,factor,fasting,ferritin,fibrin,fibrinogen,field,filtration,fl,fluid,for,form,formula,fraction,free,ft3,function,g,g/dl,g/l,globulin,glomerular,glucose,glycosylated,gm/day,gram,gravity,hb,hba1c,hbc,hbe,hbs,hcv,hdl,hdl-chol,helper,hematocrit,hemoglobin,hepatitis,high,hiv,hiv1,hormone,hour,hours,hr,human,i.cardiac,identified,identifier,ig,iga,igg,igm,immunoassay,immunoblot,immunodeficiency,immunofluorescence,immunoglobulin,immunoglobulins,impedance,in,indirect,influenza,inhibitor,inorganic,inr,international,iron,isoenzyme,isoenzymes,iu/l,iu/ml,ix,k,ketone,ketones,kinase,kinase-mb,kinase.mb,kit,lactate,lactatedehydrogenase,large,latex,ldh,ldl,ldl-chol,leptospira,leukocyte,leukocytes,level,light,limit,lipid,liver,load,log,lymphocytes,lymphocytes/100,m,m.predicted,magnesium,manual,mass,mass/time,mass/volume,mb,mch,mchc,mcv,mdrd,mean,meia,method,mg/day,mg/dl,mg/l,microscopic,microscopy,miu/ml,ml/min,ml/min/1.73,ml/min/1.73m2,mm/hr,mmol/l,moles/volume,monocytes,monocytes/100,morphology,mosm/kg,mosmol/kg,mu/ml,mycoplasma,n-terminal,na,naa,narrative,nasal,natriuretic,nec,nephelometry,neutrophils,neutrophils/100,ng/dl,ng/l,ng/ml,nitrite,nitrogen,normalize,npo,ns1,nt-pro,nuclear,nucleated,null,observation,of,on,only,or,osmolality,other,p24,panel,parathyrin.intact,parathyroid,partial,pcr,pct,peptide.b,peritoneal,pg,pg/ml,ph,pha,phosphatase,phosphate,phosphokinase,phosphorus,plasma,platelet,platelets,pleural,pmol/l,pneumoniae,po,poor,post,potassium,power,presence,probe,procalcitonin,profile,prohormone,protein,prothombin,prothrombin,pt,ptt,purple,pyruvate,qualitative,qualititative,quantitaive,quantitative,radioimmunoassay,random,rapid,rate,rate/1.73,rate/area,ratio,reaction,reactive,reagent,real,receptor,red,refractometry,renal,respiratory,reticulocyte,reticulocytes,ria,rich,rna,sars,sars-cov-2,schwartz,screening,sediment,sedimentation,sensitivity,serum,serum/plasma,sgot,sgpt,signal,slide/smear,smith,sodium,sp,specific,specimen,spinal,spp.,spun,sq,stain,stimulating,strand,strip,surface,swab,syncytial,synovial,t.cardiac,t3,t4,target,test,test.complement,tg,thomboplastin,thromboplastin,thromboplatin,thyroid,thyroxine,tibc,time,titer,to,total,triglyceride,triglycerides,triiodothyronine,troponin,tsh,type,u/l,u/ml,ug/day,ug/dl,ug/ml,umol/l,units/ml,units/volume,unsaturated,unspecified,urate,urea,uric,urine,urobilinogen,vein,venous,viii,viral,virus,vitamin,vldl,volume,westergren,white,whole,width,with,xxx,สั่งรายการเดียว
idf_weights,3.110831,7.002652,4.199291,4

In [40]:
ans = pd.concat([tf_ans, new_idf], ignore_index=True)
ans

,,#/area,#/volume,%,+,+/-,-,--1,--2,/mm3,0.5,1,1+2,1.7,1.9,10*3/ul,10*6/ul,10*9/l,10^12/l,10^3/ul,2,2.6,20,2019,24,25,400,50,75,=,?g/dl,a1c,a1c/hemoglobin.total,ab,ab+hiv1,acetylcholine,acid,activated,active,activity,activity/volume,actual/normal,aerobe,aerobic,ag,alanine,albumin,alkaline,alt,aminotransferase,amplification,anaerobe,and,anti,anti-dsdna,anti-sm,antibody,anticardiolipin,antigen,antiglobulin,antinuclear,any,appearance,aptt,arterial,aspartate,assay,ast,automate,automated,b,bacteria,bacterial,basophils,bcp,bicarbonate,bilirubin,bilirubin.albumin,bilirubin.direct,bilirubin.glucuronidated,bilirubin.total,binding,blood,bnp,body,bound,bromocresol,bun,by,c,c-reactive,c3,c4,calciferol,calcitonin,calcitriol,calcium,calculation,capacity,capillaries,capillary,cardiolipin,cd3+cd4+,cd4+cd8+,cell,cell/hpf,cells,cells/100,cells/l,cells/ml,cells/mm3,cells/ul,centrifugation,centrifuged,cerebral,chloride,chol,cholesterol,chylomicrons,ck-mb,ckd-epi,cl,clearance,clearance/1.73,co2,coagulation,collected,color,complement,complete,concentration,conjugated,control,copies/ml,cord,core,corona,coronavirus,corpuscular,cortisol,count,covid-19,cpk,creatine,creatinine,creatinine-based,crp,csf,culture,d,d+metabolites,d-dimer,dehydrogenase,dengue,deprecated,detection,differential,direct,disease,distribution,dna,double,dried,duration,dye,e,eclia,egfr,electrolyte,elisa,entitic,enzymatic,eosinophils,eosinophils/100,epithelial,erythrocyte,erythrocytes,erythrocytes/100,esr,esterase,estimates,examination,extractable,factor,fasting,ferritin,fibrin,fibrinogen,field,filtration,fl,fluid,for,form,formula,fraction,free,ft3,function,g,g/dl,g/l,globulin,glomerular,glucose,glycosylated,gm/day,gram,gravity,hb,hba1c,hbc,hbe,hbs,hcv,hdl,hdl-chol,helper,hematocrit,hemoglobin,hepatitis,high,hiv,hiv1,hormone,hour,hours,hr,human,i.cardiac,identified,identifier,ig,iga,igg,igm,immunoassay,immunoblot,immunodeficiency,immunofluorescence,immunoglobulin,immunoglobulins,impedance,in,indirect,influenza,inhibitor,inorganic,inr,international,iron,isoenzyme,isoenzymes,iu/l,iu/ml,ix,k,ketone,ketones,kinase,kinase-mb,kinase.mb,kit,lactate,lactatedehydrogenase,large,latex,ldh,ldl,ldl-chol,leptospira,leukocyte,leukocytes,level,light,limit,lipid,liver,load,log,lymphocytes,lymphocytes/100,m,m.predicted,magnesium,manual,mass,mass/time,mass/volume,mb,mch,mchc,mcv,mdrd,mean,meia,method,mg/day,mg/dl,mg/l,microscopic,microscopy,miu/ml,ml/min,ml/min/1.73,ml/min/1.73m2,mm/hr,mmol/l,moles/volume,monocytes,monocytes/100,morphology,mosm/kg,mosmol/kg,mu/ml,mycoplasma,n-terminal,na,naa,narrative,nasal,natriuretic,nec,nephelometry,neutrophils,neutrophils/100,ng/dl,ng/l,ng/ml,nitrite,nitrogen,normalize,npo,ns1,nt-pro,nuclear,nucleated,null,observation,of,on,only,or,osmolality,other,p24,panel,parathyrin.intact,parathyroid,partial,pcr,pct,peptide.b,peritoneal,pg,pg/ml,ph,pha,phosphatase,phosphate,phosphokinase,phosphorus,plasma,platelet,platelets,pleural,pmol/l,pneumoniae,po,poor,post,potassium,power,presence,probe,procalcitonin,profile,prohormone,protein,prothombin,prothrombin,pt,ptt,purple,pyruvate,qualitative,qualititative,quantitaive,quantitative,radioimmunoassay,random,rapid,rate,rate/1.73,rate/area,ratio,reaction,reactive,reagent,real,receptor,red,refractometry,renal,respiratory,reticulocyte,reticulocytes,ria,rich,rna,sars,sars-cov-2,schwartz,screening,sediment,sedimentation,sensitivity,serum,serum/plasma,sgot,sgpt,signal,slide/smear,smith,sodium,sp,specific,specimen,spinal,spp.,spun,sq,stain,stimulating,strand,strip,surface,swab,syncytial,synovial,t.cardiac,t3,t4,target,test,test.complement,tg,thomboplastin,thromboplastin,thromboplatin,thyroid,thyroxine,tibc,time,titer,to,total,triglyceride,triglycerides,triiodothyronine,troponin,tsh,type,u/l,u/ml,ug/day,ug/dl,ug/ml,umol/l,units/ml,units/volume,unsaturated,unspecified,urate,urea,uric,urine,urobilinogen,vein,venous,viii,viral,virus,vitamin,vldl,volume,westergren,white,whole,width,with,xxx,สั่งรายการเดียว
0,0.000000,0.000000,0.000000,0.000000,0.

In [41]:
a = pd.concat([tf_ans, new_idf])
a

,,#/area,#/volume,%,+,+/-,-,--1,--2,/mm3,0.5,1,1+2,1.7,1.9,10*3/ul,10*6/ul,10*9/l,10^12/l,10^3/ul,2,2.6,20,2019,24,25,400,50,75,=,?g/dl,a1c,a1c/hemoglobin.total,ab,ab+hiv1,acetylcholine,acid,activated,active,activity,activity/volume,actual/normal,aerobe,aerobic,ag,alanine,albumin,alkaline,alt,aminotransferase,amplification,anaerobe,and,anti,anti-dsdna,anti-sm,antibody,anticardiolipin,antigen,antiglobulin,antinuclear,any,appearance,aptt,arterial,aspartate,assay,ast,automate,automated,b,bacteria,bacterial,basophils,bcp,bicarbonate,bilirubin,bilirubin.albumin,bilirubin.direct,bilirubin.glucuronidated,bilirubin.total,binding,blood,bnp,body,bound,bromocresol,bun,by,c,c-reactive,c3,c4,calciferol,calcitonin,calcitriol,calcium,calculation,capacity,capillaries,capillary,cardiolipin,cd3+cd4+,cd4+cd8+,cell,cell/hpf,cells,cells/100,cells/l,cells/ml,cells/mm3,cells/ul,centrifugation,centrifuged,cerebral,chloride,chol,cholesterol,chylomicrons,ck-mb,ckd-epi,cl,clearance,clearance/1.73,co2,coagulation,collected,color,complement,complete,concentration,conjugated,control,copies/ml,cord,core,corona,coronavirus,corpuscular,cortisol,count,covid-19,cpk,creatine,creatinine,creatinine-based,crp,csf,culture,d,d+metabolites,d-dimer,dehydrogenase,dengue,deprecated,detection,differential,direct,disease,distribution,dna,double,dried,duration,dye,e,eclia,egfr,electrolyte,elisa,entitic,enzymatic,eosinophils,eosinophils/100,epithelial,erythrocyte,erythrocytes,erythrocytes/100,esr,esterase,estimates,examination,extractable,factor,fasting,ferritin,fibrin,fibrinogen,field,filtration,fl,fluid,for,form,formula,fraction,free,ft3,function,g,g/dl,g/l,globulin,glomerular,glucose,glycosylated,gm/day,gram,gravity,hb,hba1c,hbc,hbe,hbs,hcv,hdl,hdl-chol,helper,hematocrit,hemoglobin,hepatitis,high,hiv,hiv1,hormone,hour,hours,hr,human,i.cardiac,identified,identifier,ig,iga,igg,igm,immunoassay,immunoblot,immunodeficiency,immunofluorescence,immunoglobulin,immunoglobulins,impedance,in,indirect,influenza,inhibitor,inorganic,inr,international,iron,isoenzyme,isoenzymes,iu/l,iu/ml,ix,k,ketone,ketones,kinase,kinase-mb,kinase.mb,kit,lactate,lactatedehydrogenase,large,latex,ldh,ldl,ldl-chol,leptospira,leukocyte,leukocytes,level,light,limit,lipid,liver,load,log,lymphocytes,lymphocytes/100,m,m.predicted,magnesium,manual,mass,mass/time,mass/volume,mb,mch,mchc,mcv,mdrd,mean,meia,method,mg/day,mg/dl,mg/l,microscopic,microscopy,miu/ml,ml/min,ml/min/1.73,ml/min/1.73m2,mm/hr,mmol/l,moles/volume,monocytes,monocytes/100,morphology,mosm/kg,mosmol/kg,mu/ml,mycoplasma,n-terminal,na,naa,narrative,nasal,natriuretic,nec,nephelometry,neutrophils,neutrophils/100,ng/dl,ng/l,ng/ml,nitrite,nitrogen,normalize,npo,ns1,nt-pro,nuclear,nucleated,null,observation,of,on,only,or,osmolality,other,p24,panel,parathyrin.intact,parathyroid,partial,pcr,pct,peptide.b,peritoneal,pg,pg/ml,ph,pha,phosphatase,phosphate,phosphokinase,phosphorus,plasma,platelet,platelets,pleural,pmol/l,pneumoniae,po,poor,post,potassium,power,presence,probe,procalcitonin,profile,prohormone,protein,prothombin,prothrombin,pt,ptt,purple,pyruvate,qualitative,qualititative,quantitaive,quantitative,radioimmunoassay,random,rapid,rate,rate/1.73,rate/area,ratio,reaction,reactive,reagent,real,receptor,red,refractometry,renal,respiratory,reticulocyte,reticulocytes,ria,rich,rna,sars,sars-cov-2,schwartz,screening,sediment,sedimentation,sensitivity,serum,serum/plasma,sgot,sgpt,signal,slide/smear,smith,sodium,sp,specific,specimen,spinal,spp.,spun,sq,stain,stimulating,strand,strip,surface,swab,syncytial,synovial,t.cardiac,t3,t4,target,test,test.complement,tg,thomboplastin,thromboplastin,thromboplatin,thyroid,thyroxine,tibc,time,titer,to,total,triglyceride,triglycerides,triiodothyronine,troponin,tsh,type,u/l,u/ml,ug/day,ug/dl,ug/ml,umol/l,units/ml,units/volume,unsaturated,unspecified,urate,urea,uric,urine,urobilinogen,vein,venous,viii,viral,virus,vitamin,vldl,volume,westergren,white,whole,width,with,xxx,สั่งรายการเดียว
"(loinc, 62)",0.000000,0.000000,0.000000

In [42]:
a.iloc[0:808] * a.loc['idf_weights'] 

,,#/area,#/volume,%,+,+/-,-,--1,--2,/mm3,0.5,1,1+2,1.7,1.9,10*3/ul,10*6/ul,10*9/l,10^12/l,10^3/ul,2,2.6,20,2019,24,25,400,50,75,=,?g/dl,a1c,a1c/hemoglobin.total,ab,ab+hiv1,acetylcholine,acid,activated,active,activity,activity/volume,actual/normal,aerobe,aerobic,ag,alanine,albumin,alkaline,alt,aminotransferase,amplification,anaerobe,and,anti,anti-dsdna,anti-sm,antibody,anticardiolipin,antigen,antiglobulin,antinuclear,any,appearance,aptt,arterial,aspartate,assay,ast,automate,automated,b,bacteria,bacterial,basophils,bcp,bicarbonate,bilirubin,bilirubin.albumin,bilirubin.direct,bilirubin.glucuronidated,bilirubin.total,binding,blood,bnp,body,bound,bromocresol,bun,by,c,c-reactive,c3,c4,calciferol,calcitonin,calcitriol,calcium,calculation,capacity,capillaries,capillary,cardiolipin,cd3+cd4+,cd4+cd8+,cell,cell/hpf,cells,cells/100,cells/l,cells/ml,cells/mm3,cells/ul,centrifugation,centrifuged,cerebral,chloride,chol,cholesterol,chylomicrons,ck-mb,ckd-epi,cl,clearance,clearance/1.73,co2,coagulation,collected,color,complement,complete,concentration,conjugated,control,copies/ml,cord,core,corona,coronavirus,corpuscular,cortisol,count,covid-19,cpk,creatine,creatinine,creatinine-based,crp,csf,culture,d,d+metabolites,d-dimer,dehydrogenase,dengue,deprecated,detection,differential,direct,disease,distribution,dna,double,dried,duration,dye,e,eclia,egfr,electrolyte,elisa,entitic,enzymatic,eosinophils,eosinophils/100,epithelial,erythrocyte,erythrocytes,erythrocytes/100,esr,esterase,estimates,examination,extractable,factor,fasting,ferritin,fibrin,fibrinogen,field,filtration,fl,fluid,for,form,formula,fraction,free,ft3,function,g,g/dl,g/l,globulin,glomerular,glucose,glycosylated,gm/day,gram,gravity,hb,hba1c,hbc,hbe,hbs,hcv,hdl,hdl-chol,helper,hematocrit,hemoglobin,hepatitis,high,hiv,hiv1,hormone,hour,hours,hr,human,i.cardiac,identified,identifier,ig,iga,igg,igm,immunoassay,immunoblot,immunodeficiency,immunofluorescence,immunoglobulin,immunoglobulins,impedance,in,indirect,influenza,inhibitor,inorganic,inr,international,iron,isoenzyme,isoenzymes,iu/l,iu/ml,ix,k,ketone,ketones,kinase,kinase-mb,kinase.mb,kit,lactate,lactatedehydrogenase,large,latex,ldh,ldl,ldl-chol,leptospira,leukocyte,leukocytes,level,light,limit,lipid,liver,load,log,lymphocytes,lymphocytes/100,m,m.predicted,magnesium,manual,mass,mass/time,mass/volume,mb,mch,mchc,mcv,mdrd,mean,meia,method,mg/day,mg/dl,mg/l,microscopic,microscopy,miu/ml,ml/min,ml/min/1.73,ml/min/1.73m2,mm/hr,mmol/l,moles/volume,monocytes,monocytes/100,morphology,mosm/kg,mosmol/kg,mu/ml,mycoplasma,n-terminal,na,naa,narrative,nasal,natriuretic,nec,nephelometry,neutrophils,neutrophils/100,ng/dl,ng/l,ng/ml,nitrite,nitrogen,normalize,npo,ns1,nt-pro,nuclear,nucleated,null,observation,of,on,only,or,osmolality,other,p24,panel,parathyrin.intact,parathyroid,partial,pcr,pct,peptide.b,peritoneal,pg,pg/ml,ph,pha,phosphatase,phosphate,phosphokinase,phosphorus,plasma,platelet,platelets,pleural,pmol/l,pneumoniae,po,poor,post,potassium,power,presence,probe,procalcitonin,profile,prohormone,protein,prothombin,prothrombin,pt,ptt,purple,pyruvate,qualitative,qualititative,quantitaive,quantitative,radioimmunoassay,random,rapid,rate,rate/1.73,rate/area,ratio,reaction,reactive,reagent,real,receptor,red,refractometry,renal,respiratory,reticulocyte,reticulocytes,ria,rich,rna,sars,sars-cov-2,schwartz,screening,sediment,sedimentation,sensitivity,serum,serum/plasma,sgot,sgpt,signal,slide/smear,smith,sodium,sp,specific,specimen,spinal,spp.,spun,sq,stain,stimulating,strand,strip,surface,swab,syncytial,synovial,t.cardiac,t3,t4,target,test,test.complement,tg,thomboplastin,thromboplastin,thromboplatin,thyroid,thyroxine,tibc,time,titer,to,total,triglyceride,triglycerides,triiodothyronine,troponin,tsh,type,u/l,u/ml,ug/day,ug/dl,ug/ml,umol/l,units/ml,units/volume,unsaturated,unspecified,urate,urea,uric,urine,urobilinogen,vein,venous,viii,viral,virus,vitamin,vldl,volume,westergren,white,whole,width,with,xxx,สั่งรายการเดียว
"(loinc, 62)",0.000000,0.0,0.0,0.0,0.0,0

In [43]:
ans.iloc[808]

                   3.110831
#/area             7.002652
#/volume           4.199291
%                  4.805427
+                  7.002652
                     ...   
whole              4.700067
width              6.086361
with               4.560305
xxx                6.597187
สั่งรายการเดียว    7.002652
Name: 808, Length: 493, dtype: float64

In [44]:
ans = tf_ans * ans.loc[808] 
ans

#/area  #/volume    %    +  +/-        -  --1  --2  /mm3  \
loinc 62   0.000000     0.0       0.0  0.0  0.0  0.0  0.00000  0.0  0.0   0.0   
      25   0.000000     0.0       0.0  0.0  0.0  0.0  0.00000  0.0  0.0   0.0   
cdg   171  0.000000     0.0       0.0  0.0  0.0  0.0  0.00000  0.0  0.0   0.0   
tmlt  150  0.000000     0.0       0.0  0.0  0.0  0.0  0.00000  0.0  0.0   0.0   
cdg   43   0.000000     0.0       0.0  0.0  0.0  0.0  0.00000  0.0  0.0   0.0   
...             ...     ...       ...  ...  ...  ...      ...  ...  ...   ...   
loinc 132  0.000000     0.0       0.0  0.0  0.0  0.0  0.00000  0.0  0.0   0.0   
tmlt  192  0.000000     0.0       0.0  0.0  0.0  0.0  0.00000  0.0  0.0   0.0   
cdg   142  0.000000     0.0       0.0  0.0  0.0  0.0  0.00000  0.0  0.0   0.0   
      72   0.345648     0.0       0.0  0.0  0.0  0.0  0.51164  0.0  0.0   0.0   
      197  0.518472     0.0       0.0  0.0  0.0  0.0  0.00000  0.0  0.0   0.0   

           0.5         1  1+2  1.7  1.9  10*3/ul  10*6/ul  10*9/l  10^12/l  \
loinc 62   0.0  0.000000  0.0  0.0  0.0      0.0      0.0     0.0      0.0   
      25   0.0  0.000000  0.0  0.0  0.0      0.0      0.0     0.0      0.0   
cdg   171  0.0  0.000000  0.0  0.0  0.0      0.0      0.0     0.0      0.0   
tmlt  150  0.0  0.266363  0.0  0.0  0.0      0.0      0.0     0.0      0.0   
cdg   43   0.0  0.000000  0.0  0.0  0.0      0.0      0.0     0.0      0.0   
...        ...       ...  ...  ...  ...      ...      ...     ...      ...   
loinc 132  0.0  0.000000  0.0  0.0  0.0      0.0      0.0     0.0      0.0   
tmlt  192  0.0  0.000000  0.0  0.0  0.0      0.0      0.0     0.0      0.0   
cdg   142  0.0  0.000000  0.0  0.0  0.0      0.0      0.0     0.0      0.0   
      72   0.0  0.000000  0.0  0.0  0.0      0.0      0.0     0.0      0.0   
      197  0.0  0.000000  0.0  0.0  0.0      0.0      0.0     0.0      0.0   

           10^3/ul    2  2.6   20  2019   24   25  400   50   75    =  ?g/dl  \
loinc 62       0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0   
      25       0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0   
cdg   171      0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0   
tmlt  150      0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0   
cdg   43       0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0   
...            ...  ...  ...  ...   ...  ...  ...  ...  ...  ...  ...    ...   
loinc 132      0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0   
tmlt  192      0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0   
cdg   142      0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0   
      72       0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0   
      197      0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0   

           a1c  a1c/hemoglobin.total        ab  ab+hiv1  acetylcholine  acid  \
loinc 62   0.0                   0.0  0.000000      0.0            0.0   0.0   
      25   0.0                   0.0  0.000000      0.0            0.0   0.0   
cdg   171  0.0                   0.0  0.000000      0.0            0.0   0.0   
tmlt  150  0.0                   0.0  0.000000      0.0            0.0   0.0   
cdg   43   0.0                   0.0  0.000000      0.0            0.0   0.0   
...        ...                   ...       ...      ...            ...   ...   
loinc 132  0.0                   0.0  0.000000      0.0            0.0   0.0   
tmlt  192  0.0                   0.0  0.420316      0.0            0.0   0.0   
cdg   142  0.0                   0.0  0.000000      0.0            0.0   0.0   
      72   0.0                   0.0  0.000000      0.0            0.0   0.0   
      197  0.0                   0.0  0.000000      0.0            0.0   0.0   

           activated  active  activity  activity/volume  actual/normal  \
loinc 62         0.0     0.0       0.0              0.0            0.0   
      25         0.0     0.0       0.0   

In [ ]:
from matplotlib import pyplot as plt
y_pred_tfidf
y_train
# plt.scatter(y_train, y_pred_tfidf)

In [ ]:
dfTest = pd.read_excel('/content/drive/MyDrive/unique-lab.xlsx', index_col=0)
dfTest

In [144]:
Splitinput = dfTest.loc[:, ('TestName')].str.split(r"[\([{ });,.\]]+")
Splitinput = Splitinput.fillna('Null')
Splitinput.sample(10)

702132                                 [CD4]
289314                                 [FT4]
539679                          [KOH, Mount]
497125                      [Corrected, WBC]
282178           [Alpha, Fetoprotein, AFP, ]
107960                         [Hypochromia]
368981                                  [PT]
359587    [RSV, Respiratory, Syncytial, Vir]
25593                               [Result]
293307                        [Syphilis, TP]
Name: TestName, dtype: object

In [153]:

# Builds a dictionary of features and transforms documents to feature vectors and convert our text documents to a
# matrix of token counts (CountVectorizer)
count_vect = CountVectorizer(tokenizer= lambda x:x, lowercase = False)
X_train_counts = count_vect.fit_transform(x_train)

# transform a count matrix to a normalized tf-idf representation (tf-idf transformer)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)


knn = KNeighborsClassifier(n_neighbors=5)

# training our classifier ; train_data.target will be having numbers assigned for each category in train data
clf = knn.fit(X_train_tfidf, y_train)

# Input Data to predict their classes of the given categories
# docs_new = ['I have a Harley Davidson and Yamaha.', 'I have a GTX 1050 GPU']
# building up feature vector of our input
X_new_counts = count_vect.transform(Splitinput)
# We call transform instead of fit_transform because it's already been fit
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

# predicting the category of our input text: Will give out number for category
predicted = clf.predict(X_new_tfidf)

for doc, category in zip(Splitinput, predicted):
  # print(category)
    print('%r => %s' % (doc, category))



['BUN'] => Ascites Profile;White Blood Cell
['Creatinine'] => Ascites Profile;White Blood Cell
['eGFR'] => Ascites Profile;White Blood Cell
['Specimen'] => Ascites Profile;White Blood Cell
['Sodium'] => Ascites Profile;White Blood Cell
['Potassium'] => Ascites Profile;White Blood Cell
['Chloride'] => Ascites Profile;White Blood Cell
['Total', 'CO2'] => Ascites Profile;White Blood Cell
['Anion', 'Gap'] => Ascites Profile;White Blood Cell
['Albumin'] => Ascites Profile;White Blood Cell
['Hb'] => Ascites Profile;White Blood Cell
['Hct'] => Ascites Profile;White Blood Cell
['WBC'] => Ascites Profile;White Blood Cell
['Corrected', 'WBC'] => Ascites Profile;White Blood Cell
['Differential', 'Cell', 'Count'] => Ascites Profile;White Blood Cell
['Blast'] => Ascites Profile;White Blood Cell
['Promyelocyte'] => Ascites Profile;White Blood Cell
['Myelocyte'] => Ascites Profile;White Blood Cell
['Meta', 'Myelocyte'] => Ascites Profile;White Blood Cell
['Band', 'Form'] => Ascites Profile;White Bloo